<a href="https://colab.research.google.com/github/yf591/Using-stable-diffusion-webui-in-Google-Colab/blob/main/stable_diffusion_webui_AUTOMATIC1111_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ◆モデルのダウンロードからする場合
- Hugging Faceからのダウンロードは並列処理
- Civitaiからのダウンロードは並列処理だとエラーがでるため個別処理

## 事前準備

In [ ]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# --- 設定 ---
# Stable Diffusion Web UI のディレクトリ
webui_dir = "/content/stable-diffusion-webui"
# モデルを保存するディレクトリ
model_dir = f"{webui_dir}/models/Stable-diffusion"
vae_dir = f"{webui_dir}/models/VAE"
lora_dir = f"{webui_dir}/models/Lora"
embedding_dir = f"{webui_dir}/embeddings"
extension_dir = f"{webui_dir}/extensions"

In [ ]:
import os

# Stable Diffusion Web UI のクローン
if not os.path.exists(webui_dir):
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else:
    print("stable-diffusion-webui already exists")

# ディレクトリの作成
for dir_path in [model_dir, vae_dir, lora_dir, embedding_dir, extension_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created.")

In [ ]:
from google.colab import output

# --- 依存関係のインストール ---
!pip install -r {webui_dir}/requirements_versions.txt
!pip install -U xformers
!pip install torch==2.5.0+cu124 torchvision==0.16.0+cu124 torchaudio==2.5.0+cu124 torchtext==0.16.0 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu124
!pip install wandb==0.15.12
# !pip install aria2
!pip install -U requests
!pip install basicsr # 一旦ここに置く

output.clear()

In [ ]:
# --- セットアップ ---
from google.colab import userdata
from huggingface_hub import login

# HF_TOKEN が存在する場合、ログイン
HF_TOKEN = os.environ.get("HF_TOKEN")
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("Successfully logged in using environment variable.")

# CIVITAI_TOKEN が存在する場合、取得
api_key = userdata.get('CIVITAI_TOKEN')
if api_key is None:
    print("Error: CIVITAI_API_KEY secret is not set.")

## 関数の定義（ダウンロードの設定）

In [ ]:
import urllib.request
from concurrent.futures import ThreadPoolExecutor
import subprocess
import time
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import requests

def download_file_with_wget(url, path):
    """wget を使用してファイルをダウンロードする"""
    if not os.path.exists(path):
        print(f"Downloading: {os.path.basename(path)}...")
        try:
            process = subprocess.Popen(['wget', url, '-O', path],
                                       stdout=subprocess.PIPE,
                                       stderr=subprocess.PIPE)
            stdout, stderr = process.communicate()
            if stderr and stderr.decode().strip():
                print(f"Error downloading {os.path.basename(path)}:\n{stderr.decode()}")
                return False
        except Exception as e:
            print(f"Error downloading {os.path.basename(path)}: {e}")
            return False
        print(f"{os.path.basename(path)} download completed")
        return True
    else:
        print(f"{os.path.basename(path)} is already downloaded")
        return True


def get_file_size(url):
    """URLからファイルのサイズを取得する"""
    try:
        response = requests.head(url, allow_redirects=True)
        response.raise_for_status()
        size = response.headers.get('content-length')
        if size is not None:
            return int(size)
        else:
            return "不明"
    except Exception:
        return "不明"

def download_file(file_info, progress_bar):
    """ファイル情報を元にダウンロードまたはコピーを行う"""
    path = file_info["path"]
    if not os.path.exists(path):
        print(f"Downloading: {os.path.basename(path)}...")
        if file_info["source"] == "local":
            try:
              !cp {file_info["local_path"]} {path}
              progress_bar.value = 100
            except Exception as e:
                print(f"Error copying local file {os.path.basename(path)}: {e}")
                return
        elif file_info["source"] == "url":
            try:
              url = file_info["url"]
              if url.startswith("https://civitai.com/api/download/models"):
                    if download_file_with_wget(url, path) == False:
                       return
              else:
                 file_size = get_file_size(url)
                 if file_size == "不明":
                    print(f"File size: Unknown")
                 else:
                    print(f"File size: {file_size / (1024 * 1024):.2f} MB")

                 def download_progress(block_num, block_size, total_size):
                     if total_size > 0:
                         percentage = (block_num * block_size) * 100 // total_size
                         progress_bar.value = percentage
                 urllib.request.urlretrieve(url, path, reporthook = download_progress)
            except Exception as e:
                print(f"Error downloading {os.path.basename(path)}: {e}")
                return
        else:
            print(f"Error: Invalid source: {file_info['source']}")
            return
        print(f"{os.path.basename(path)} download completed")
    else:
        print(f"{os.path.basename(path)} is already downloaded")


def download_files_threaded(files_info, category_name):
    """ファイル情報を元にスレッドを使って並列にダウンロードまたはコピーを行う"""
    print(f"{category_name} downloads start...")
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for key, file_info in files_info.items():
            if file_info["checked"]: # チェックされていない場合はスキップ
                  progress_bar = widgets.IntProgress(value=0, min=0, max=100, description=f'{key[:20]}...', orientation='horizontal')
                  display(progress_bar)
                  future = executor.submit(download_file, file_info, progress_bar)
                  futures.append(future)
        for future in futures:
            future.result()
    print(f"{category_name} downloads finished.")

    # --- UIの作成 ---
def create_checkboxes(choices, description):
    print(description)
    checkboxes = []
    for key, value in choices.items():
        checkbox = widgets.Checkbox(value=value["checked"], description=key, layout=widgets.Layout(width='auto'))
        hbox = widgets.HBox([checkbox], layout=widgets.Layout(width='auto'))
        checkboxes.append(hbox)
    return checkboxes

def update_choices(choices, checkboxes):
    for key, checkbox_container in zip(choices.keys(), checkboxes):
        checkbox = checkbox_container.children[0]
        choices[key]["checked"] = checkbox.value

# CSSを適用
display(HTML("""
<style>
    .widget-checkbox label {
        white-space: nowrap;
    }
</style>
"""))

## BaseModelのダウンロード
※Civitaiからのモデルのダウンロードは個別の対応が必要

In [ ]:
#@title ### **Base Model用（Hugging Face & Local）**

# 使用するモデルとダウンロードURLを辞書型で定義
model_choices = {
   #@markdown ---
   #@markdown ## **実写系モデル(Base: SD 1.5)** / From Hugging Face

    #@markdown ### ◆ [AbsoluteRealityV1.6525.safetensors](https://huggingface.co/Lykon/AbsoluteReality/blob/main/AbsoluteRealityV1.6525.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/LlMlnkorC_k)
    "AbsoluteReality v1.6": {
        "source": "url",
        "url": "https://huggingface.co/Lykon/AbsoluteReality/resolve/main/AbsoluteRealityV1.6525.safetensors",
        "path": f"{model_dir}/AbsoluteRealityV1.6525.safetensors",
        "checked": False
    },
   #@markdown ### ◆ [AbsoluteReality_1.8.1_pruned.safetensors](https://huggingface.co/Lykon/AbsoluteReality/blob/main/AbsoluteReality_1.8.1_pruned.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/vUr99l5UReA)
    "AbsoluteReality v1.8.1": {
        "source": "url",
        "url": "https://huggingface.co/Lykon/AbsoluteReality/resolve/main/AbsoluteReality_1.8.1_pruned.safetensors",
        "path": f"{model_dir}/AbsoluteReality_1.8.1_pruned.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [BRAV5finalfp16.safetensors](https://huggingface.co/BanKaiPls/AsianModel/blob/main/BRAV5finalfp16.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/g0i1_LSzrWQ)
     "BRAV5": {
        "source": "url",
        "url": "https://huggingface.co/BanKaiPls/AsianModel/resolve/main/BRAV5finalfp16.safetensors",
        "path": f"{model_dir}/BRAV5finalfp16.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [Brav6.safetensors](https://huggingface.co/BanKaiPls/AsianModel/blob/main/Brav6.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/vcQplWWdyEk)
     "BRAV6": {
        "source": "url",
        "url": "https://huggingface.co/BanKaiPls/AsianModel/resolve/main/Brav6.safetensors",
        "path": f"{model_dir}/Brav6.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [BracingEvoMix_v1.safetensors](https://huggingface.co/sazyou-roukaku/BracingEvoMix/blob/main/BracingEvoMix_v1.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/G6mZUq3S354)
    "BracingEvoMix(v1)": {
        "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/BracingEvoMix/resolve/main/BracingEvoMix_v1.safetensors",
        "path": f"{model_dir}/BracingEvoMix_v1.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [BracingEvoMix_v2.safetensors](https://huggingface.co/sazyou-roukaku/BracingEvoMix/blob/main/BracingEvoMix_v2.safetensors)
    "BracingEvoMix(v2)": {
        "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/BracingEvoMix/resolve/main/BracingEvoMix_v2.safetensors",
        "path": f"{model_dir}/BracingEvoMix_v2.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [BracingEvoMix_Another_v1.safetensors](https://huggingface.co/sazyou-roukaku/BracingEvoMix/blob/main/BracingEvoMix_Another_v1.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/hdQxapP2Y98)
   "BracingEvoMix - Another": {
        "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/BracingEvoMix/resolve/main/BracingEvoMix_Another_v1.safetensors",
        "path": f"{model_dir}/BracingEvoMix_Another_v1.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [BracingEvoMix_Fast_v1.safetensors](https://huggingface.co/sazyou-roukaku/BracingEvoMix/blob/main/BracingEvoMix_Fast_v1.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/hdQxapP2Y98)
     "BracingEvoMix - Fast": {
        "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/BracingEvoMix/resolve/main/BracingEvoMix_Fast_v1.safetensors",
        "path": f"{model_dir}/BracingEvoMix_Fast_v1.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [chilled_remix_v2.safetensors](https://huggingface.co/sazyou-roukaku/chilled_remix/blob/main/chilled_remix_v2.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/c5N1zq6cSNE)
    "chilled_remix v2": {
         "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors",
        "path": f"{model_dir}/chilled_remix_v2.safetensors",
         "checked": False
    },
    #@markdown ### ◆ [chilled_reversemix_v2.safetensors](https://huggingface.co/sazyou-roukaku/chilled_remix/blob/main/chilled_reversemix_v2.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/c5N1zq6cSNE)
      "chilled_reversemix v2": {
        "source": "url",
        "url": "https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_reversemix_v2.safetensors",
        "path": f"{model_dir}/chilled_reversemix_v2.safetensors",
         "checked": False
    },
     #@markdown ### ◆ [DreamShaper_7_pruned.safetensors](https://huggingface.co/Lykon/DreamShaper/blob/main/DreamShaper_7_pruned.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/u071zv1lxug)
     "DreamShaper v7": {
        "source": "url",
        "url": "https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_7_pruned.safetensors",
        "path": f"{model_dir}/DreamShaper_7_pruned.safetensors",
         "checked": False
    },
    #@markdown ### ◆ [DreamShaper_8_pruned.safetensors](https://huggingface.co/Lykon/DreamShaper/blob/main/DreamShaper_8_pruned.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/gIfmr12C4X8)
    "DreamShaper v8": {
        "source": "url",
        "url": "https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors",
        "path": f"{model_dir}/DreamShaper_8_pruned.safetensors",
         "checked": False
    },
        #@markdown ### ◆ [majicmixRealistic_v6.safetensors](https://huggingface.co/casque/majicmixRealistic_v6/blob/main/majicmixRealistic_v6.safetensors)
    "majicmixRealistic_v6": {
        "source": "url",
        "url": "https://huggingface.co/casque/majicmixRealistic_v6/resolve/main/majicmixRealistic_v6.safetensors",
        "path": f"{model_dir}/majicmixRealistic_v6.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [OpenBra.safetensors](https://huggingface.co/BanKaiPls/AsianModel/blob/main/OpenBra.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/0ZK7O5EFqWA?si=nU1tMkXFv-cW6ZMA)
    "OpenBra": {
        "source": "url",
        "url": "https://huggingface.co/BanKaiPls/AsianModel/resolve/main/OpenBra.safetensors",
        "path": f"{model_dir}/OpenBra.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [sd-v1-5-pruned-noema-fp16.safetensors](https://huggingface.co/hollowstrawberry/stable-diffusion-guide/blob/main/models/sd-v1-5-pruned-noema-fp16.safetensors)
    "sd-v1-5-pruned-noema-fp16": {
        "source": "url",
        "url": "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors",
        "path": f"{model_dir}/sd-v1-5-pruned-noema-fp16.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [models/R1311/VoidnoiseCore_R1311.fp16.safetensors](https://huggingface.co/konapieces/VoidnoiseCore/blob/main/models/R1311/VoidnoiseCore_R1311.fp16.safetensors)
    "VoidnoiseCore(R1311)": {
        "source": "url",
        "url": "https://huggingface.co/konapieces/VoidnoiseCore/resolve/main/models/R1311/VoidnoiseCore_R1311.fp16.safetensors",
        "path": f"{model_dir}/VoidnoiseCore_R1311.fp16.safetensors",
        "checked": False
    },
     #@markdown ### ◆ [model/weddingImperial.safetensors](https://huggingface.co/teasan/WeddingImperial/blob/main/model/weddingImperial.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/4dukNcNJ_fI)
     "WeddingImperial v1": {
        "source": "url",
        "url": "https://huggingface.co/teasan/WeddingImperial/resolve/main/model/weddingImperial.safetensors",
        "path": f"{model_dir}/weddingImperial.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [model/weddingImperialV2.safetensors](https://huggingface.co/teasan/WeddingImperial/blob/main/model/weddingImperialV2.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/rBGYRNmznk8)
     "WeddingImperial v2": {
        "source": "url",
        "url": "https://huggingface.co/teasan/WeddingImperial/resolve/main/model/weddingImperialV2.safetensors",
         "path": f"{model_dir}/weddingImperialV2.safetensors",
        "checked": False
    },
   #@markdown ### ◆ [yayoi_mix_v20-fp16.safetensors](https://huggingface.co/Kotajiro/yayoi_mix/blob/main/yayoi_mix_v20-fp16.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/OM678ZSygow)
    "yayoi_mix v2": {
        "source": "url",
        "url": "https://huggingface.co/Kotajiro/yayoi_mix/resolve/main/yayoi_mix_v20-fp16.safetensors",
        "path": f"{model_dir}/yayoi_mix_v20-fp16.safetensors",
        "checked": False
    },

    #@markdown ---
    #@markdown ## **実写系モデル (Base: SD 2.1)** / From Hugging Face

    #@markdown ### ◆ [manga-diffusion-poc/poc.safetensors](https://huggingface.co/aipicasso/manga-diffusion-poc/blob/main/poc.safetensors)
     "Manga Diffusion PoC": {
        "source": "url",
        "url": "https://huggingface.co/aipicasso/manga-diffusion-poc/resolve/main/poc.safetensors",
        "path": f"{model_dir}/poc.safetensors",
        "checked": False
    },

    #@markdown ---
    #@markdown ## **実写系モデル (Base: SDXL)** / From Hugging Face

    #@markdown ### ◆ [fuduki_mix_v20.safetensors](https://huggingface.co/Kotajiro/fuduki_mix/blob/main/fuduki_mix_v20.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/O2GNFwB2Ncg)
    "fuduki_mix (v2.0)": {
        "source": "url",
         "url": "https://huggingface.co/Kotajiro/fuduki_mix/resolve/main/fuduki_mix_v20.safetensors",
        "path":  f"{model_dir}/fuduki_mix_v20.safetensors",
        "checked": False
    },
   #@markdown ### ◆ [diffusion_pytorch_model.fp16.safetensors](https://huggingface.co/latent-consistency/lcm-ssd-1b/blob/main/diffusion_pytorch_model.fp16.safetensors)
    "Latent Consistency Model (LCM): SSD-1B": {
        "source": "url",
        "url": "https://huggingface.co/latent-consistency/lcm-ssd-1b/resolve/main/diffusion_pytorch_model.fp16.safetensors",
        "path": f"{model_dir}/diffusion_pytorch_model.fp16.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [sd_xl_base_1.0.safetensors](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors)
    #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/IyXvmlhjjgM)
    "SDXL base 1.0": {
        "source": "url",
        "url": "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors",
        "path": f"{model_dir}/sd_xl_base_1.0.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [sdxl_lightning_1step_x0.safetensors](https://huggingface.co/ByteDance/SDXL-Lightning/blob/main/sdxl_lightning_1step_x0.safetensors)
    "SDXL-Lightning (1step)": {
        "source": "url",
        "url": "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_1step_x0.safetensors",
         "path": f"{model_dir}/sdxl_lightning_1step_x0.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [sdxl_lightning_2step.safetensors](https://huggingface.co/ByteDance/SDXL-Lightning/blob/main/sdxl_lightning_2step.safetensors)
    "SDXL-Lightning (2step)": {
        "source": "url",
        "url": "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_2step.safetensors",
         "path": f"{model_dir}/sdxl_lightning_2step.safetensors",
        "checked": False
    },
     #@markdown ### ◆ [sdxl_lightning_4step.safetensors](https://huggingface.co/ByteDance/SDXL-Lightning/blob/main/sdxl_lightning_4step.safetensors)
    "SDXL-Lightning (4step)": {
        "source": "url",
        "url": "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_4step.safetensors",
        "path": f"{model_dir}/sdxl_lightning_4step.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [sdxl_lightning_8step.safetensors](https://huggingface.co/ByteDance/SDXL-Lightning/blob/main/sdxl_lightning_8step.safetensors)
    "SDXL-Lightning (8step)": {
        "source": "url",
        "url": "https://huggingface.co/ByteDance/SDXL-Lightning/resolve/main/sdxl_lightning_8step.safetensors",
         "path": f"{model_dir}/sdxl_lightning_8step.safetensors",
         "checked": False
    },

  #@markdown ---
  #@markdown ## **イラスト系モデル (Base: SDXL)** / From Hugging Face

    #@markdown ### ◆ [models/S0964/LastpieceCoreXL_S0964.fp16.safetensors](https://huggingface.co/konapieces/LastpieceCoreXL/blob/main/models/S0964/LastpieceCoreXL_S0964.fp16.safetensors) ※VAE用コードで専用VAEをダウンロードする

    "LastpieceCoreXL": {
         "source": "url",
        "url": "https://huggingface.co/konapieces/LastpieceCoreXL/resolve/main/models/S0964/LastpieceCoreXL_S0964.fp16.safetensors",
         "path": f"{model_dir}/LastpieceCoreXL_S0964.fp16.safetensors",
         "checked": False
     },
      #@markdown ### ◆ [emi-2.safetensors](https://huggingface.co/aipicasso/emi-2/blob/main/emi-2.safetensors)
    "Emi 2": {
       "source": "url",
       "url": "https://huggingface.co/aipicasso/emi-2/resolve/main/emi-2.safetensors",
       "path": f"{model_dir}/emi-2.safetensors",
        "checked": False
    },
      #@markdown ---
      #@markdown ## **セミリアル系モデル (Base: SD 1.5)** / From Hugging Face
     #@markdown ### ◆ [LittleStepMix_v1.safetensors](https://huggingface.co/sazyou-roukaku/LittleStepMix/blob/main/LittleStepMix_v1.safetensors) ※VAE用コードで専用VAEをダウンロードする

    #@markdown LittleStepMixを導入する際は併せて、【sr_SDv2vae_kl-f8anime2.safetensors】というVAEも一緒にインストールします

   #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/a2X3oJB55nE)
     "LittleStepMix v1": {
        "source":"url",
        "url":"https://huggingface.co/sazyou-roukaku/LittleStepMix/resolve/main/LittleStepMix_v1.safetensors",
        "path":f"{model_dir}/LittleStepMix_v1.safetensors",
         "checked": False
    },

    #@markdown ---
    #@markdown ## **refiner向けモデル (SDXL向け)** / From Hugging Face
    #@markdown ### ◆ [sd_xl_refiner_1.0.safetensors](https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/blob/main/sd_xl_refiner_1.0.safetensors)

   #@markdown [生成される画像を確認する(YouTube)](https://youtu.be/CNhrbc8VRSk?si=OSQVldOp7FMU--zL)
    "SD-XL 1.0-refiner": {
        "source": "url",
        "url":"https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors",
        "path": f"{model_dir}/sd_xl_refiner_1.0.safetensors",
        "checked": False
    },
    #@markdown ---

     # From MyDrive(ローカルにDLしたBaseModel)
     "anyloraCheckpoint_bakedvaeBlessedFp16": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/anyloraCheckpoint_bakedvaeBlessedFp16.safetensors", # ローカルのファイルパスを指定
        "path": f"{model_dir}/anyloraCheckpoint_bakedvaeBlessedFp16.safetensors",
        "checked": True
     },
     "grapefruitHentaiModel_grapefruitv41": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/grapefruitHentaiModel_grapefruitv41.safetensors", # ローカルのファイルパスを指定
        "path": f"{model_dir}/grapefruitHentaiModel_grapefruitv41.safetensors",
        "checked": True
     },
     "hentaimodel_v10": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/hentaimodel_v10.safetensors", # ローカルのファイルパスを指定
        "path": f"{model_dir}/hentaimodel_v10.safetensors",
        "checked": True
     },
     "pikasNewGeneration_v20": {
       "source": "local",
       "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/pikasNewGeneration_v20.safetensors", # ローカルのファイルパスを指定
        "path": f"{model_dir}/pikasNewGeneration_v20.safetensors",
        "checked": True
     },
    # From MyDrive(自作したBaseModel)
    "merged_model_chillre_majic": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/merged_model_chillre_majic.safetensors",
        "path": f"{model_dir}/merged_model_chillre_majic.safetensors",
        "checked": True
    }
}


model_checkboxes = create_checkboxes(model_choices, "ベースモデルを選択してください")
display(widgets.VBox(model_checkboxes))

def on_model_button_clicked(b):
    clear_output()
    display(widgets.VBox(model_checkboxes))
    update_choices(model_choices, model_checkboxes)
    start_time = time.time()
    download_files_threaded(model_choices, "ベースモデル")
    end_time = time.time()
    print(f"すべての処理が完全に完了しました！({end_time - start_time:.2f}秒)")

model_button_layout = widgets.Layout(width='auto')  # ボタンの幅を自動調整
model_button = widgets.Button(description="ベースモデルをダウンロード", layout=model_button_layout)
model_button.on_click(on_model_button_clicked)
display(model_button)

In [ ]:
#@title ### **Civitaiからモデルをダウンロード（一部VAEも含む）**

import os


#@markdown ※ CivitaiからのDLするベースモデルにチェックを入れてください

#@markdown ## **実写系モデル(Base: SD 1.5) / From Civitai**

#@markdown ### ◆ [BellyUse](https://civitai.com/models/211708)

use_belly_use = False #@param {type: "boolean"}
if use_belly_use:
  !wget https://civitai.com/api/download/models/265163?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Blessing Mix](https://civitai.com/models/94179/blessing-mix-aka-bracing-evo-mix-clone)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/hNU8CMaewqI)
use_blessing_mix = False #@param {type: "boolean"}
if use_blessing_mix:
  !wget https://civitai.com/api/download/models/100468?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [BlazingRealDrive](https://civitai.com/models/137909)
use_blazing_real_drive = False #@param {type: "boolean"}
if use_blazing_real_drive:
  !wget https://civitai.com/api/download/models/202466?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [BRAV4](https://civitai.com/models/25494?modelVersionId=51395)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/t6Nm0AUyqyU)
use_bra_v4 = False #@param {type: "boolean"}
if use_bra_v4:
  !wget https://civitai.com/api/download/models/51395?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [BRAV7](https://civitai.com/models/25494?modelVersionId=177164)
use_bra_v7 = False #@param {type: "boolean"}
if use_bra_v7:
  !wget https://civitai.com/api/download/models/177164?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [CyberRealistic](https://civitai.com/models/15003/cyberrealistic)
use_cyber_realistic = False #@param {type: "boolean"}
if use_cyber_realistic:
  !wget https://civitai.com/api/download/models/198401?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # CyberRealistic Negativeも併せてダウンロード
  !wget https://civitai.com/api/download/models/82745?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [epiCRealism](https://civitai.com/models/25694?modelVersionId=134065)
use_epi_c_realism = False #@param {type: "boolean"}
if use_epi_c_realism:
  !wget https://civitai.com/api/download/models/134065?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [ICBINP](https://civitai.com/models/28059/icbinp-i-cant-believe-its-not-photography)
use_icbinp = False #@param {type: "boolean"}
if use_icbinp:
  !wget https://civitai.com/api/download/models/121557?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [CBINP - LCM](https://civitai.com/models/28059/icbinp-i-cant-believe-its-not-photography)
use_icbinp_lcm = False #@param {type: "boolean"}
if use_icbinp_lcm:
  !wget https://civitai.com/api/download/models/253668?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Kawaii Realistic Asian Mix](https://civitai.com/models/83766/kawaii-realistic-asian-mix)
use_kawaii_realistic_asian_mix = False #@param {type: "boolean"}
if use_kawaii_realistic_asian_mix:
  !wget https://civitai.com/api/download/models/170762?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [kisaragi_mix v22](https://civitai.com/models/45757/kisaragimix)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/p1zvB-aXKaw)
use_kisaragi_mix_v22 = False #@param {type: "boolean"}
if use_kisaragi_mix_v22:
  !wget https://civitai.com/api/download/models/97186?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [majicMIX realistic(v2.5)](https://civitai.com/models/43331/majicmix-realistic)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/A9Gx28_r1RE)
use_majicmix_realistic_v6 = False #@param {type: "boolean"}
if use_majicmix_realistic_v6:
  !wget https://civitai.com/api/download/models/126470?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Meichidark_Mix](https://civitai.com/models/69158/meichidarkmix)
use_meichidark_mix = False #@param {type: "boolean"}
if use_meichidark_mix:
  !wget https://civitai.com/api/download/models/147184?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/


#@markdown ### ◆ [MUSE_v1](https://civitai.com/models/13564/musev1)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/Rc6v_2rOICA)
use_muse_v1 = False #@param {type: "boolean"}
if use_muse_v1:
  !wget https://civitai.com/api/download/models/15980?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Noble Mix Fix](https://civitai.com/models/65480/noble-mix-fix)
use_noble_mix_fix = False #@param {type: "boolean"}
if use_noble_mix_fix:
  !wget https://civitai.com/api/download/models/92828?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Photon](https://civitai.com/models/84728/photon)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/eIYkdfiGx-Y)
use_photon = False #@param {type: "boolean"}
if use_photon:
  !wget https://civitai.com/api/download/models/90072?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### [RealBeautyMix](https://civitai.com/models/85156)
use_real_beauty_mix = False #@param {type: "boolean"}
if use_real_beauty_mix:
  !wget https://civitai.com/api/download/models/90505?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [RealCartoon-Realistic](https://civitai.com/models/97744/realcartoon-realistic)
use_realcartoon_realistic = False #@param {type: "boolean"}
if use_realcartoon_realistic:
  !wget https://civitai.com/api/download/models/535328?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Realistic Stock Photo](https://civitai.com/models/139565?modelVersionId=524032)
use_realistic_stock_photo = False #@param {type: "boolean"}
if use_realistic_stock_photo:
  !wget https://civitai.com/api/download/models/524032?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Realisian](https://civitai.com/models/47130/realisian)
use_realisian = False #@param {type: "boolean"}
if use_realisian:
  !wget https://civitai.com/api/download/models/325142?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [RumbleXL](https://civitai.com/models/296650/rumblexl-animagine-based-photographic-model)
use_rumblexl = False #@param {type: "boolean"}
if use_rumblexl:
  !wget https://civitai.com/api/download/models/400175?token={api_key} --content-disposition  --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Soda Mix](https://civitai.com/models/47507/soda-mix)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/2QGdRREYMoo)
use_soda_mix = False #@param {type: "boolean"}
if use_soda_mix:
  !wget https://civitai.com/api/download/models/52797?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [SweetMuse](https://civitai.com/models/81668/sweetmuse)
use_sweet_muse = False #@param {type: "boolean"}
if use_sweet_muse:
  !wget https://civitai.com/api/download/models/89839?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [t3](https://civitai.com/models/110053/t3)
use_t3 = False #@param {type: "boolean"}
if use_t3:
  !wget https://civitai.com/api/download/models/281273?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [XXMix_9realistic](https://civitai.com/models/47274/xxmix9realistic)
use_xxmix_9realistic = False #@param {type: "boolean"}
if use_xxmix_9realistic:
  !wget https://civitai.com/api/download/models/102222?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ---
#@markdown ## **実写系モデル (Base: SDXL)** / From Civitai

#@markdown ### ◆ [AfterRealXL](https://civitai.com/models/150212/afterrealxl)
use_after_real_xl = False #@param {type: "boolean"}
if use_after_real_xl:
  !wget https://civitai.com/api/download/models/167881?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Copax TimeLessXL - SDXL1.0](https://civitai.com/models/118111/copax-timelessxl-sdxl10)
use_copax_timelessxl_sdxl10 = False #@param {type: "boolean"}
if use_copax_timelessxl_sdxl10:
  !wget https://civitai.com/api/download/models/172160?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [CyberRealistic XL](https://civitai.com/models/312530)
use_cyber_realistic_xl = False #@param {type: "boolean"}
if use_cyber_realistic_xl:
  !wget https://civitai.com/api/download/models/395041?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [DreamShaper XL1.0](https://civitai.com/models/112902)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/UAUhscKUvp8)
use_dream_shaper_xl_1 = False #@param {type: "boolean"}
if use_dream_shaper_xl_1:
  !wget https://civitai.com/api/download/models/126688?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [DreamShaper XL (Lightning DPM++ SDE)](https://civitai.com/models/112902?modelVersionId=354657)
use_dream_shaper_xl_lightning_dpm_sde = False #@param {type: "boolean"}
if use_dream_shaper_xl_lightning_dpm_sde:
  !wget https://civitai.com/api/download/models/354657?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Imaginarium (Turbo boosted)](https://civitai.com/models/249973/imaginarium?modelVersionId=316617)
use_imaginarium_turbo_boosted = False #@param {type: "boolean"}
if use_imaginarium_turbo_boosted:
  !wget https://civitai.com/api/download/models/316617?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Juggernaut XL (V9+RDPhoto2-Lightning_4S)](https://civitai.com/models/133005/juggernaut-xl)
use_juggernaut_xl_v9_rdphoto2_lightning_4s = False #@param {type: "boolean"}
if use_juggernaut_xl_v9_rdphoto2_lightning_4s:
  !wget https://civitai.com/api/download/models/357609?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Muchen-SDXL-Portrait](https://civitai.com/models/450086/muchen-sdxl-portrait)
use_muchen_sdxl_portrait = False #@param {type: "boolean"}
if use_muchen_sdxl_portrait:
  !wget https://civitai.com/api/download/models/501157?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [NightVision XL](https://civitai.com/models/128607/nightvision-xl-photorealistic-or-portrait-or-photography-or-hyperreal-or-architecture-or-interior-design-no-refiner-needed)
use_nightvision_xl = False #@param {type: "boolean"}
if use_nightvision_xl:
  !wget https://civitai.com/api/download/models/159982?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [RealVisXL V4.0 Lightning (BakedVAE)](https://civitai.com/models/139562/realvisxl-v40)
use_real_vis_xl_v4_lightning_bakedvae = False #@param {type: "boolean"}
if use_real_vis_xl_v4_lightning_bakedvae:
  !wget https://civitai.com/api/download/models/361593?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [tsubaki_mix](https://civitai.com/models/455220/tsubakimix)
use_tsubaki_mix = False #@param {type: "boolean"}
if use_tsubaki_mix:
  !wget https://civitai.com/api/download/models/649263?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ---
#@markdown ## **実写系モデル (Base: Pony)** / From Civitai

#@markdown ### ◆ [AoiReal Pony](https://civitai.com/models/481162/aoireal-pony) ※VAEも併せて導入(SDXL_VAE)
use_aoireal_pony = False #@param {type: "boolean"}
if use_aoireal_pony:
  !wget https://civitai.com/api/download/models/559460?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [AutHuman Pony V4](https://civitai.com/models/477246/authuman-pony-v4) ※VAEも併せて導入(SDXL_VAE)
use_authuman_pony_v4 = False #@param {type: "boolean"}
if use_authuman_pony_v4:
  !wget https://civitai.com/api/download/models/579902?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors


#@markdown ### ◆ [Beeble's Realistic PonyXL](https://civitai.com/models/520415/beebles-realistic-ponyxl) ※VAEも併せて導入(SDXL_VAE)
use_beebles_realistic_ponyxl = False #@param {type: "boolean"}
if use_beebles_realistic_ponyxl:
  !wget https://civitai.com/api/download/models/578234?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [CG Realistic Pony](https://civitai.com/models/518021/maplemixpony) ※VAEも併せて導入(SDXL_VAE)
use_cg_realistic_pony = False #@param {type: "boolean"}
if use_cg_realistic_pony:
  !wget https://civitai.com/api/download/models/575238?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [DucHaiten-Pony-Real](https://civitai.com/models/477851/duchaiten-pony-real) ※VAEも併せて導入(SDXL_VAE)
use_ducHaiten_pony_real = False #@param {type: "boolean"}
if use_ducHaiten_pony_real:
  !wget https://civitai.com/api/download/models/619549?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [MapleMix_Pony](https://civitai.com/models/518021/maplemixpony) ※VAEも併せて導入(SDXL_VAE)
use_maplemix_pony = False #@param {type: "boolean"}
if use_maplemix_pony:
  !wget https://civitai.com/api/download/models/575622?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors


#@markdown ### ◆ [MinkiePie](https://civitai.com/models/475514/minkiepie-pony-mix) ※VAEも併せて導入(SDXL_VAE)
use_minkiePie = False #@param {type: "boolean"}
if use_minkiePie:
  !wget https://civitai.com/api/download/models/528902?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [Pony Diffusion V6 XL](https://civitai.com/models/257749) ※VAEも併せて導入(SDXL_VAE)
use_pony_diffusion_v6_xl = False #@param {type: "boolean"}
if use_pony_diffusion_v6_xl:
    !wget "https://civitai.com/api/download/models/290640?type=Model&format=SafeTensor&size=pruned&fp=fp16&token={api_key}" --output-document=/content/stable-diffusion-webui/models/Stable-diffusion/ponyDiffusionV6XL_v6StartWithThisOne.safetensors
    # VAEも併せて導入(SDXL_VAE)
    !mkdir -p /content/stable-diffusion-webui/models/VAE/
    !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [Pony Realism](https://civitai.com/models/372465/pony-realism) ※VAEも併せて導入(SDXL_VAE)
use_pony_realism = False #@param {type: "boolean"}
if use_pony_realism:
  !wget https://civitai.com/api/download/models/534642?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [realPony_JPDoll](https://civitai.com/models/420600) ※VAEも併せて導入(SDXL_VAE)
use_realpony_jpdoll = False #@param {type: "boolean"}
if use_realpony_jpdoll:
  !wget https://civitai.com/api/download/models/468687?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [WAI-REAL_CN](https://civitai.com/models/469902/wai-realcn) ※VAEも併せて導入(SDXL_VAE)
use_wai_real_cn = False #@param {type: "boolean"}
if use_wai_real_cn:
  !wget https://civitai.com/api/download/models/632685?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ---
#@markdown ## **イラスト系モデル (Base: SD 1.5)** / From Civitai

#@markdown ### ◆ [AnyLoRA - Checkpoint](https://civitai.com/models/23900/anylora-checkpoint) ※ローカルにDL済み
use_anylora_checkpoint = False #@param {type: "boolean"}
if use_anylora_checkpoint:
  !wget https://civitai.com/api/download/models/95489?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [BraveDawn](https://civitai.com/models/101932)
#@markdown VAEの `vae-ft-mse-840000-ema-pruned.ckpt` を併せて利用することが公式より推奨されています
use_brave_dawn = False #@param {type: "boolean"}
if use_brave_dawn:
  !wget https://civitai.com/api/download/models/128200?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Break the Darkness](https://civitai.com/models/105047/break-the-darkness)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/Slpt9bQ6884)
use_break_the_darkness = False #@param {type: "boolean"}
if use_break_the_darkness:
  !wget https://civitai.com/api/download/models/123633?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [ENCartoony](https://civitai.com/models/143113)
use_encartoony = False #@param {type: "boolean"}
if use_encartoony:
  !wget https://civitai.com/api/download/models/158785?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [ENCLight-v1](https://civitai.com/models/161452/enclight-v1)
use_enclight_v1 = False #@param {type: "boolean"}
if use_enclight_v1:
  !wget https://civitai.com/api/download/models/181723?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Flat-2D Animerge](https://civitai.com/models/35960/flat-2d-animerge)
use_flat2d_animerge = False #@param {type: "boolean"}
if use_flat2d_animerge:
  !wget https://civitai.com/api/download/models/266360?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [meinamix](https://civitai.com/models/7240/meinamix)
#@markdown [生成される画像を確認する(YouTube)](https://youtu.be/LX7_3vmUImY)
use_meina_mix = False #@param {type: "boolean"}
if use_meina_mix:
  !wget https://civitai.com/api/download/models/119057?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [TMND-Mix](https://civitai.com/models/27259/tmnd-mix)
use_tmnd_mix = False #@param {type: "boolean"}
if use_tmnd_mix:
  !wget https://civitai.com/api/download/models/221220?type=Model&format=SafeTensor&size=pruned&fp=fp16?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ---
#@markdown ## **イラスト系モデル (Base: SDXL)** / From Civitai

#@markdown ### ◆ [AAM_XL_AnimeMix](https://civitai.com/models/269232?modelVersionId=303526)
use_aam_xl_animemix = False #@param {type: "boolean"}
if use_aam_xl_animemix:
  !wget https://civitai.com/api/download/models/303526?type=Model&format=SafeTensor&size=full&fp=fp16?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/


#@markdown ### ◆ [Animagine XL V3](https://civitai.com/models/260267/animagine-xl-v3)
use_animagine_xl_v3 = False #@param {type: "boolean"}
if use_animagine_xl_v3:
  !wget https://civitai.com/api/download/models/293564?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [blue_pencil-XL](https://civitai.com/models/119012/bluepencil-xl) ※推奨されるNegative Embeddingsも導入
use_blue_pencil_xl = False #@param {type: "boolean"}
if use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/212090?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # 推奨されるNegative Embeddingsも導入
  # unaestheticXL | Negative TI (_Jug6, _Sky3.1, _AYv1)
  # URL: https://civitai.com/models/119032/unaestheticxl-or-negative-ti)
  !wget https://civitai.com/api/download/models/207934?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/
  !wget https://civitai.com/api/download/models/175819?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/
  !wget https://civitai.com/api/download/models/162146?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [blue_pencil-XL-LCM](https://civitai.com/models/202108/bluepencil-xl-lcm)
use_blue_pencil_xl_lcm = False #@param {type: "boolean"}
if use_blue_pencil_xl_lcm:
  !wget https://civitai.com/api/download/models/227693?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Illustrious-XL](https://civitai.com/models/795765/illustrious-xl)
use_illustrious_xl = False #@param {type: "boolean"}
if use_illustrious_xl:
  !wget https://civitai.com/api/download/models/889818?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Kohaku-XL alpha](https://civitai.com/models/136389/kohaku-xl-alpha)
use_kohaku_xl_alpha = False #@param {type: "boolean"}
if use_kohaku_xl_alpha:
  !wget https://civitai.com/api/download/models/167926?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [NekoRayXL](https://civitai.com/models/136719/nekorayxl)
use_neko_ray_xl = False #@param {type: "boolean"}
if use_neko_ray_xl:
  !wget https://civitai.com/api/download/models/150826?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [NoobAI-XL (NAI-XL)](https://civitai.com/models/833294/noobai-xl-nai-xl)
use_noobai_xl = False #@param {type: "boolean"}
if use_noobai_xl:
  !wget https://civitai.com/api/download/models/1116447?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/

#@markdown ### ◆ [Shiroverse XL](https://civitai.com/models/378088/shiroverse-xl)
use_shiroverse_xl = False #@param {type: "boolean"}
if use_shiroverse_xl:
  !wget https://civitai.com/api/download/models/422177?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/


#@markdown ---
#@markdown ## **イラスト系モデル (Base: Pony)** / From Civitai

#@markdown ### ◆ [AutismMix SDXL](https://civitai.com/models/288584/autismmix-sdxl) ※VAEも併せて導入(SDXL_VAE)
use_autismmix_sdxl = False #@param {type: "boolean"}
if use_autismmix_sdxl:
  !mkdir -p /content/stable-diffusion-webui-forge/models/Stable-diffusion/
  !wget "https://civitai.com/api/download/models/324619?token={civitai_api_key}" --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui-forge/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={civitai_api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [MomoiroPony](https://civitai.com/models/316882/momoiropony) ※VAEも併せて導入(SDXL_VAE)
use_momoiropony = False #@param {type: "boolean"}
if use_momoiropony:
  !mkdir -p /content/stable-diffusion-webui-forge/models/Stable-diffusion/
  !wget "https://civitai.com/api/download/models/382513?token={civitai_api_key}" --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui-forge/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={civitai_api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [Pony Diffusion V6 XL](https://civitai.com/models/257749) ※VAEも併せて導入(SDXL_VAE)
use_pony_diffusion_v6_xl = False #@param {type: "boolean"}
if use_pony_diffusion_v6_xl:
  !mkdir -p /content/stable-diffusion-webui/models/Stable-diffusion/
  !wget "https://civitai.com/api/download/models/290640?type=Model&format=SafeTensor&size=pruned&fp=fp16&token={civitai_api_key}" --output-document=/content/stable-diffusion-webui/models/Stable-diffusion/ponyDiffusionV6XL_v6StartWithThisOne.safetensors
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={civitai_api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors

#@markdown ### ◆ [T-ponynai3](https://civitai.com/models/317902) ※VAEも併せて導入(SDXL_VAE)
use_t_ponynai3 = False #@param {type: "boolean"}
if use_t_ponynai3:
  !mkdir -p /content/stable-diffusion-webui-forge/models/Stable-diffusion/
  !wget "https://civitai.com/api/download/models/593760?token={civitai_api_key}" --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/Stable-diffusion/
  # VAEも併せて導入(SDXL_VAE)
  !mkdir -p /content/stable-diffusion-webui-forge/models/VAE/
  !wget "https://civitai.com/api/download/models/290640?type=VAE&format=SafeTensor&token={civitai_api_key}" --output-document=/content/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors


## 各種モデルのダウンロード（Lora, VAE, Embedding）

In [ ]:
#@title ### **Lora用（From Hugging Face & Local）**

#@markdown ## **Loras** ※基本的にはローカルにDLして使用する
#@markdown ---

# Loraファイルのコピー先
lora_choices = {

    #@markdown ## **Lora from Hugging face（Base: SD 1.5）**
    #@markdown ## ※現時点では無し
    # #@markdown ### ◆ Loraの名前をここに入力
    # "Loraの名前をここに入力": {
    #     "source": "url",
    #     "url": "https://huggingface.co/",
    #     "path": f"{lora_dir}/Loraのファイル名.safetensors",
    #     "checked": False
    # },
    #@markdown ---
    #@markdown ## **自作Lora（Base: SD 1.5）**
    #@markdown ### ◆ AmazonPhotos_Sn
    "AmazonPhotos_Sn-15": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_Sn/output/AmazonPhotos_Sn-15.safetensors",
         "path": f"{lora_dir}/AmazonPhotos_Sn-15.safetensors",
        "checked": True
    },
     "AmazonPhotos_Sn-20": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_Sn/output/AmazonPhotos_Sn-20.safetensors",
         "path": f"{lora_dir}/AmazonPhotos_Sn-20.safetensors",
         "checked": True
    },
     "AmazonPhotos_Sn-25": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_Sn/output/AmazonPhotos_Sn-25.safetensors",
         "path": f"{lora_dir}/AmazonPhotos_Sn-25.safetensors",
          "checked": True
    },

    #@markdown ### ◆ AmazonPhotos_2_Sn
    "AmazonPhotos_2_Sn-15": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_2_Sn/output/AmazonPhotos_2_Sn-15.safetensors",
         "path": f"{lora_dir}/AmazonPhotos_2_Sn-15.safetensors",
         "checked": True
    },
    "AmazonPhotos_2_Sn-20": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_2_Sn/output/AmazonPhotos_2_Sn-20.safetensors",
         "path": f"{lora_dir}/AmazonPhotos_2_Sn-20.safetensors",
        "checked": True
    },
   "AmazonPhotos_2_Sn-25": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/AmazonPhotos_2_Sn/output/AmazonPhotos_2_Sn-25.safetensors",
        "path": f"{lora_dir}/AmazonPhotos_2_Sn-25.safetensors",
        "checked": True
    },

    #@markdown ### ◆ AmazonPhotos_2_Sn
    "CuteGirl_Mix_Sn_Lora-05": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix_Sn_Lora/output/CuteGirl_Mix_Sn_Lora-05.safetensors",
         "path": f"{lora_dir}/CuteGirl_Mix_Sn_Lora-05.safetensors",
         "checked": True
    },

    #@markdown ### ◆ CuteGirl_Mix_Sn_Lora
     "CuteGirl_Mix_Sn_Lora-10": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix_Sn_Lora/output/CuteGirl_Mix_Sn_Lora-10.safetensors",
         "path": f"{lora_dir}/CuteGirl_Mix_Sn_Lora-10.safetensors",
         "checked": True
    },
    "CuteGirl_Mix_Sn_Lora-15": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix_Sn_Lora/output/CuteGirl_Mix_Sn_Lora-15.safetensors",
          "path": f"{lora_dir}/CuteGirl_Mix_Sn_Lora-15.safetensors",
         "checked": True
    },
    "CuteGirl_Mix2_Sn_Lora-10": {
         "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix2_Sn_Lora/output/CuteGirl_Mix2_Sn_Lora-10.safetensors",
          "path": f"{lora_dir}/CuteGirl_Mix2_Sn_Lora-10.safetensors",
         "checked": True
    },
    "CuteGirl_Mix2_Sn_Lora-15": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix2_Sn_Lora/output/CuteGirl_Mix2_Sn_Lora-15.safetensors",
          "path": f"{lora_dir}/CuteGirl_Mix2_Sn_Lora-15.safetensors",
         "checked": True
    },
    "CuteGirl_Mix2_Sn_Lora-20": {
        "source": "local",
        "local_path":"/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirl_Mix2_Sn_Lora/output/CuteGirl_Mix2_Sn_Lora-20.safetensors",
        "path": f"{lora_dir}/CuteGirl_Mix2_Sn_Lora-20.safetensors",
        "checked": True
    },
     "CuteGirlMix1-2Lora_Merged_lora": {
        "source": "local",
         "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/CuteGirlMix1-2Lora_Merged_lora.safetensors",
         "path": f"{lora_dir}/CuteGirlMix1-2Lora_Merged_lora.safetensors",
         "checked": True
     },

    #@markdown ### ◆ all_fours
    "all_fours-10": {
        "source": "local",
        "local_path":"/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/all_fours/output/all_fours-10.safetensors",
        "path": f"{lora_dir}/all_fours-10.safetensors",
        "checked": True
    },
    "all_fours-15": {
       "source": "local",
       "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/all_fours/output/all_fours-15.safetensors",
        "path": f"{lora_dir}/all_fours-15.safetensors",
        "checked": True
    },
     "all_fours-20": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/all_fours/output/all_fours-20.safetensors",
        "path": f"{lora_dir}/all_fours-20.safetensors",
         "checked": True
    },

    #@markdown ---
    #@markdown ## **DowndloadしたLora（Base: SD 1.5）**
    #@markdown ### ◆ [PovGroupSex_v10](https://civitai.com/models/114843/pov-group-sex-or-sex-with-multiple-girls)
     "PovGroupSex_v10": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/PovGroupSex_v10.safetensors",
        "path":  f"{lora_dir}/PovGroupSex_v10.safetensors",
        "checked": True
    },
    #@markdown ### ◆ [MultipleGirlsGroup](https://civitai.com/models/51136/multiple-girls-group)
    "MultipleGirlsGroup": {
        "source": "local",
       "local_path":  "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/MultipleGirlsGroup.safetensors",
        "path": f"{lora_dir}/MultipleGirlsGroup.safetensors",
        "checked": True
    },
    #@markdown ### ◆ [style21](https://civitai.com/models/135903/anime-style-box)
     "style21": {
        "source": "local",
       "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/style21.safetensors",
        "path": f"{lora_dir}/style21.safetensors",
         "checked": True
     },
    #@markdown ### ◆ [animemix_v3_offset](https://civitai.com/models/4982/anime-screencap-style-lora)
     "animemix_v3_offset": {
        "source": "local",
        "local_path":  "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/animemix_v3_offset.safetensors",
        "path": f"{lora_dir}/animemix_v3_offset.safetensors",
        "checked": True
    },
    #@markdown ### ◆ [eropose_allfours](https://civitai.com/models/95579/eropose-all-fours)
    "eropose_allfours": {
        "source": "local",
        "local_path": "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/eropose_allfours.safetensors",
         "path":  f"{lora_dir}/eropose_allfours.safetensors",
         "checked": True
     }
 }

lora_checkboxes = create_checkboxes(lora_choices, "Loraを選択してください")
display(widgets.VBox(lora_checkboxes))

def on_lora_button_clicked(b):
    clear_output()
    display(widgets.VBox(lora_checkboxes))
    update_choices(lora_choices, lora_checkboxes)
    start_time = time.time()
    download_files_threaded(lora_choices, "LoRA")
    end_time = time.time()
    print(f"すべての処理が完全に完了しました！({end_time - start_time:.2f}秒)")

lora_button_layout = widgets.Layout(width='auto')  # ボタンの幅を自動調整
lora_button = widgets.Button(description="LoRAをダウンロード（またはMyDriveからコピー）", layout=lora_button_layout)
lora_button.on_click(on_lora_button_clicked)
display(lora_button)

In [ ]:
#@title ### **VAE用（From Hugging Face & Local）**

#@markdown ## **VAE**
#@markdown ---
#@markdown　※導入するVAEの選択肢

# VAEファイルの情報
vae_choices = {
    #@markdown ### ◆ [vae-ft-mse-840000](https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.safetensors)
    "vae-ft-mse-840000": {
         "source": "url",
        "url": "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors",
        "path": f"{vae_dir}/vae-ft-mse-840000-ema-pruned.safetensors",
         "checked": False
    },

    #@markdown ### ◆ [kl-f8-anime2](https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime2.ckpt)
    "kl-f8-anime2": {
         "source": "url",
        "url": "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt",
        "path": f"{vae_dir}/kl-f8-anime2.ckpt",
        "checked": False
    },

    #@markdown ### ◆ [SDXL - VAE (SDXL向け)](https://huggingface.co/stabilityai/sdxl-vae/blob/main/sdxl_vae.safetensors)
    "sdxl_vae": {
         "source": "url",
        "url": "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors",
        "path": f"{vae_dir}/sdxl_vae.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [LastpieceCoreXL用の VAE](https://huggingface.co/konapieces/LastpieceCoreXL/blob/main/vae/lastpiece.xl.vae.safetensors)
    "LastpieceCoreXL VAE": {
        "source": "url",
        "url": "https://huggingface.co/konapieces/LastpieceCoreXL/resolve/main/vae/lastpiece.xl.vae.safetensors",
        "path": f"{vae_dir}/lastpiece.xl.vae.safetensors",
        "checked": False
    },
    #@markdown ### ◆ [LittleStepMix用の VAE](https://huggingface.co/sazyou-roukaku/LittleStepMix/blob/main/VAE/sr_SDv2vae_kl-f8anime2.safetensors)
    "LittleStepMix VAE": {
        "source": "url",
        "url":"https://huggingface.co/sazyou-roukaku/LittleStepMix/resolve/main/VAE/sr_SDv2vae_kl-f8anime2.safetensors",
        "path": f"{vae_dir}/sr_SDv2vae_kl-f8anime2.safetensors",
        "checked": False
    }
}

vae_checkboxes = create_checkboxes(vae_choices, "VAEを選択してください")
display(widgets.VBox(vae_checkboxes))


def on_vae_button_clicked(b):
    clear_output()
    display(widgets.VBox(vae_checkboxes))
    update_choices(vae_choices, vae_checkboxes)
    start_time = time.time()
    download_files_threaded(vae_choices, "VAE")
    end_time = time.time()
    print(f"すべての処理が完全に完了しました！({end_time - start_time:.2f}秒)")

vae_button_layout = widgets.Layout(width='auto')  # ボタンの幅を自動調整
vae_button = widgets.Button(description="VAEをダウンロード", layout=vae_button_layout)
vae_button.on_click(on_vae_button_clicked)
display(vae_button)

In [ ]:
#@title ### **Embedding用（From Hugging Face & Local）**

#@markdown ## **embeddings**
#@markdown ---
#@markdown　※導入するEmbeddingの選択肢

# Embedding ファイルの情報
embedding_choices = {
    #@markdown ### ◆ [EasyNegative](https://huggingface.co/datasets/gsdf/EasyNegative/blob/main/EasyNegative.safetensors)
    "EasyNegative": {
        "source": "url",
        "url": "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors",
        "path": f"{embedding_dir}/EasyNegative.safetensors",
        "checked": True
    },
}


embedding_checkboxes = create_checkboxes(embedding_choices, "Embeddingを選択してください")

display(widgets.VBox(embedding_checkboxes))


def on_embedding_button_clicked(b):
    clear_output()
    display(widgets.VBox(embedding_checkboxes))
    update_choices(embedding_choices, embedding_checkboxes)
    start_time = time.time()
    download_files_threaded(embedding_choices, "Embedding")
    end_time = time.time()
    print(f"すべての処理が完全に完了しました！({end_time - start_time:.2f}秒)")

embedding_button_layout = widgets.Layout(width='auto')  # ボタンの幅を自動調整
embedding_button = widgets.Button(description="Embeddingをダウンロード", layout=embedding_button_layout)
embedding_button.on_click(on_embedding_button_clicked)
display(embedding_button)

In [ ]:
#@title ### **Embedding用（From Civitai）**

#@markdown ## **embeddings**
#@markdown ---
#@markdown ※導入したいembeddingsにチェックを入れてください

#@markdown ### ◆ [negative_hand Negative Embedding](https://civitai.com/models/56519/negativehand-negative-embedding)
use_negative_hand_negative_embedding = True #@param {type: "boolean"}
if use_negative_hand_negative_embedding:
  !wget https://civitai.com/api/download/models/60938?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [badhandv4](https://civitai.com/models/16993/badhandv4-animeillustdiffusion)
use_badhandv4 = True #@param {type: "boolean"}
if use_badhandv4:
  !wget https://civitai.com/api/download/models/20068?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [unaestheticXL | Negative TI(_Jug6)](https://civitai.com/models/119032/unaestheticxl-or-negative-ti)
use_unaesthetic_xl_jug6 = False #@param {type: "boolean"}
if use_unaesthetic_xl_jug6 and not use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/207934?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [unaestheticXL | Negative TI(_Sky3.1)](https://civitai.com/models/119032/unaestheticxl-or-negative-ti)
use_unaesthetic_xl_sky31 = False #@param {type: "boolean"}
if use_unaesthetic_xl_sky31 and not use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/175819?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [unaestheticXL | Negative TI(_AYv1)](https://civitai.com/models/119032/unaestheticxl-or-negative-ti)
use_unaesthetic_xl_ayv1 = False #@param {type: "boolean"}
if use_unaesthetic_xl_ayv1 and not use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/162146?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [unaestheticXL | Negative TI(_Alb2)](https://civitai.com/models/119032/unaestheticxl-or-negative-ti)
use_unaesthetic_xl_alb2 = False #@param {type: "boolean"}
if use_unaesthetic_xl_alb2 and not use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/363593?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

#@markdown ### ◆ [Pony PDXL Negative Embeddings](https://civitai.com/models/332646/pony-pdxl-negative-embeddings)
use_pony_pdxl_negative_embeddings = False #@param {type: "boolean"}
if use_pony_pdxl_negative_embeddings and not use_blue_pencil_xl:
  !wget https://civitai.com/api/download/models/720175?token={civitai_api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/embeddings/

In [ ]:
#@title ### **Upscale用**

#@markdown ## **Upscale**
#@markdown ---

#@markdown ### ◆ [4x-UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

#@markdown #### - ダウンロードについて

#@markdown こちらはMegaにファイルがホストされている関係でプログラム側から直接ダウンロードが出来ません。[4x-UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)より直接ダウンロードした上で、/content/stable-diffusion-webui/models/ESRGANにファイルを格納してご利用ください。


#@markdown ### ◆ [4x-AnimeSharp](https://openmodeldb.info/models/4x-AnimeSharp)


#@markdown #### - ダウンロードについて

#@markdown こちらはMegaにファイルがホストされている関係でプログラム側から直接ダウンロードが出来ません。[4x-AnimeSharp](https://openmodeldb.info/models/4x-AnimeSharp)より直接ダウンロードした上で、/content/stable-diffusion-webui/models/ESRGANにファイルを格納してご利用ください。

## ControlNet用
まずGit hubから[sd-webui-controlnet](https://github.com/Mikubill/sd-webui-controlnet.git) がインストールされ、Hugging FaceからControlNetモデル、IP-Adapter-FaceID用のLoRAがダウンロードされてmodels/ControlNet に配置されます。

In [ ]:
#@title ### **ControlNetモデルのダウンロード**

#@markdown ## **ControlNet**
#@markdown ---

#@markdown ※導入したいControlNetにチェックを入れてください

import os
import urllib.request

# --- 設定 ---
# Stable Diffusion Web UI のディレクトリ
webui_dir = "/content/stable-diffusion-webui"
extension_dir = f"{webui_dir}/extensions"
controlnet_model_dir = f"{webui_dir}/models/ControlNet"  # 正しいControlNetモデルのディレクトリ
lora_dir = f"{webui_dir}/models/Lora"

# ControlNetをインストールした状態でweb UIを実行すると"ERROR: Failed building wheel for pycairo"というエラーが出るため、以下のコマンドをまずは実施
!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
# IP-Adapter-FaceIDを利用する際に必要になる
!pip install insightface==0.7.3

# ControlNet 拡張機能のインストール (既に存在するか確認)
if not os.path.exists(f"{extension_dir}/sd-webui-controlnet"):
    print("ControlNet拡張機能をインストールします...")
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git {extension_dir}/sd-webui-controlnet
    # requirements.txt がある場合はインストール
    if os.path.exists(f"{extension_dir}/sd-webui-controlnet/requirements.txt"):
        !pip install -r {extension_dir}/sd-webui-controlnet/requirements.txt
    print("ControlNet拡張機能のインストールが完了しました")
else:
    print("ControlNet拡張機能は既にインストール済みです")


# IP-Adapter-FaceID用のLoRAを格納するため、先にディレクトリを作成
!mkdir -p {lora_dir}

#@markdown ### ◆ **SD 1.5 用 ControlNet モデル**
#@markdown 以下のモデルは、SD 1.5 のベースモデルで使用できます。
use_control_net_sd15 = False #@param {type: "boolean"}
if use_control_net_sd15:
    # modelファイルのダウンロード
    model_urls = {
        "control_v11e_sd15_ip2p.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
        "control_v11e_sd15_shuffle.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
        "control_v11f1e_sd15_tile.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth",
        "control_v11f1p_sd15_depth.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
        "control_v11p_sd15_canny.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
        "control_v11p_sd15_inpaint.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
        "control_v11p_sd15_lineart.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
        "control_v11p_sd15_mlsd.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
        "control_v11p_sd15_normalbae.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
        "control_v11p_sd15_openpose.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
        "control_v11p_sd15_scribble.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
        "control_v11p_sd15_seg.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
        "control_v11p_sd15_softedge.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth",
        "control_v11p_sd15s2_lineart_anime.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth",
        "ioclab_sd15_recolor.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors",
        "ip-adapter_sd15.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth",
        "ip-adapter_sd15_plus.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth",
        "control_sd15_inpaint_depth_hand_fp16.safetensors": "https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors", # コード1でDLするもの
    }
    for filename, url in model_urls.items():
        model_path = os.path.join(controlnet_model_dir, filename)
        if not os.path.exists(model_path):
            print(f"{filename} をダウンロードします...")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {controlnet_model_dir} -o {filename}
            print("ダウンロードが完了しました")
        else:
            print(f"{filename} は既にダウンロード済みです")
     # IP-Adapter-FaceID
    face_id_model_urls = {
        "ip-adapter-faceid_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin",
        "ip-adapter-faceid-plus_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin",
        "ip-adapter-faceid-plusv2_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin",
        "ip-adapter-faceid-portrait_sd15.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sd15.bin",
        "ip-adapter-faceid_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors",
        "ip-adapter-faceid-plus_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors",
        "ip-adapter-faceid-plusv2_sd15_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors"
    }

    for filename, url in face_id_model_urls.items():
         if filename.endswith(".safetensors"):
            model_path = os.path.join(lora_dir, filename)
         else:
            model_path = os.path.join(controlnet_model_dir, filename)
         if not os.path.exists(model_path):
            print(f"{filename} をダウンロードします...")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {os.path.dirname(model_path)} -o {os.path.basename(model_path)}
            print("ダウンロードが完了しました")
         else:
            print(f"{filename} は既にダウンロード済みです")

#@markdown ### ◆ **SDXL 用 ControlNet モデル**
#@markdown 以下のモデルは、SDXL のベースモデルで使用できます。
use_control_net_sdxl = False #@param {type: "boolean"}
if use_control_net_sdxl:
    # modelファイルのダウンロード
    sdxl_model_urls = {
        "diffusers_xl_canny_full.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors",
        "diffusers_xl_canny_mid.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors",
        "diffusers_xl_canny_small.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_small.safetensors",
        "diffusers_xl_depth_full.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors",
        "diffusers_xl_depth_mid.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors",
        "diffusers_xl_depth_small.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_small.safetensors",
        "kohya_controllllite_xl_canny_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors",
        "kohya_controllllite_xl_depth_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors",
        "kohya_controllllite_xl_openpose_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime.safetensors",
        "kohya_controllllite_xl_openpose_anime_v2.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors",
        "kohya_controllllite_xl_scribble_anime.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors",
        "sai_xl_canny_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_128lora.safetensors",
        "sai_xl_canny_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors",
        "sai_xl_depth_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_128lora.safetensors",
        "sai_xl_depth_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors",
        "sai_xl_recolor_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors",
        "sai_xl_recolor_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors",
        "sai_xl_sketch_128lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_128lora.safetensors",
        "sai_xl_sketch_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors",
        "sargezt_xl_depth.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth.safetensors",
        "sargezt_xl_depth_faid_vidit.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_faid_vidit.safetensors",
        "sargezt_xl_depth_zeed.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_zeed.safetensors",
        "sargezt_xl_softedge.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors",
        "t2i-adapter_xl_canny.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors",
        "t2i-adapter_xl_openpose.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors",
        "t2i-adapter_xl_sketch.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors",
        "thibaud_xl_openpose.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors",
        "thibaud_xl_openpose_256lora.safetensors": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors",
        "ip-adapter_xl.pth": "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth",
    }

    for filename, url in sdxl_model_urls.items():
        model_path = os.path.join(controlnet_model_dir, filename)
        if not os.path.exists(model_path):
            print(f"{filename} をダウンロードします...")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {controlnet_model_dir} -o {filename}
            print("ダウンロードが完了しました")
        else:
            print(f"{filename} は既にダウンロード済みです")
    # IP-Adapter-FaceID
    face_id_xl_model_urls = {
        "ip-adapter-faceid_sdxl.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin",
        "ip-adapter-faceid-plusv2_sdxl.bin": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin",
        "ip-adapter-faceid_sdxl_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors",
        "ip-adapter-faceid-plusv2_sdxl_lora.safetensors": "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors",
    }
    for filename, url in face_id_xl_model_urls.items():
        if filename.endswith(".safetensors"):
           model_path = os.path.join(lora_dir, filename)
        else:
           model_path = os.path.join(controlnet_model_dir, filename)

        if not os.path.exists(model_path):
             print(f"{filename} をダウンロードします...")
             !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {os.path.dirname(model_path)} -o {os.path.basename(model_path)}
             print("ダウンロードが完了しました")
        else:
            print(f"{filename} は既にダウンロード済みです")

     # Instant ID
    instant_id_model_urls = {
        "ip-adapter_instant_id_sdxl.bin": "https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin",
        "control_instant_id_sdxl.safetensors": "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors"
    }
    for filename, url in instant_id_model_urls.items():
        model_path = os.path.join(controlnet_model_dir, filename)
        if not os.path.exists(model_path):
             print(f"{filename} をダウンロードします...")
             !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {controlnet_model_dir} -o {filename}
             print("ダウンロードが完了しました")
        else:
             print(f"{filename} は既にダウンロード済みです")

print("すべての処理が完了しました。")

## Extensionのダウンロード

In [ ]:
#@title ### **Extension用**

#@markdown ## **Extension**
#@markdown ---
#@markdown ※導入したいExtensionにチェックを入れてください

# --- 設定 ---
# Stable Diffusion Web UI のディレクトリ
webui_dir = "/content/stable-diffusion-webui"
extension_dir = f"{webui_dir}/extensions"
lora_dir = f"{webui_dir}/models/Lora"


# 拡張機能をインストール（直接git clone）
print("Extensionのダウンロードを開始します...")

#@markdown ### ★ [After Detailer](https://github.com/Bing-su/adetailer)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/WJfhNjzjMIE)
use_after_detailer = True #@param {type: "boolean"}
if use_after_detailer:
  !git clone https://github.com/Bing-su/adetailer.git {extension_dir}/adetailer
  # ADetailer (After Detailer) Lips Model (URL: https://civitai.com/models/142240/adetailer-after-detailer-lips-model)
  !mkdir -p /content/stable-diffusion-webui/models/adetailer
  !wget https://civitai.com/api/download/models/157700?token={api_key} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/adetailer
  !unzip /content/stable-diffusion-webui/models/adetailer/adetailerAfterDetailer_v10.zip -d /content/stable-diffusion-webui/models/adetailer/

#@markdown ### ★ [AnimateDiff](https://github.com/continue-revolution/sd-webui-animatediff)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/jzt8LXczMb4)
use_animate_diff = False #@param {type: "boolean"}
if use_animate_diff:
  # AnimateDiffの導入
  !git clone https://github.com/continue-revolution/sd-webui-animatediff.git {extension_dir}/sd-webui-animatediff

#@markdown 以下にチェックをつけると、AnimateDiff用のmodelはv15_v3のみをダウンロードするようにします。チェックをつけたほうがセットアップ処理の時間削減に繋がります。（v15_v2のみのチェックボックスと併用可能です）
only_use_animate_diff_v15_v3 = False #@param {type: "boolean"}
if only_use_animate_diff_v15_v3:
  # AnimateDiff用のモーションモジュールダウンロード (v15_v3)
  !wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix={extension_dir}/sd-webui-animatediff/model

#@markdown 以下にチェックをつけると、AnimateDiff用のmodelはv15_v2のみをダウンロードするようにします。チェックをつけたほうがセットアップ処理の時間削減に繋がります。（v15_v3のみのチェックボックスと併用可能です）
only_use_animate_diff_v15_v2 = False #@param {type: "boolean"}
if only_use_animate_diff_v15_v2:
  # AnimateDiff用のモーションモジュールダウンロード (v15_v2)
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

if only_use_animate_diff_v15_v3 is False and only_use_animate_diff_v15_v2 is False and use_animate_diff is True:
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v14.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/guoyww/animatediff/resolve/refs%2Fpr%2F3/mm_sd_v15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model
  !wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model

#@markdown ### ★ [Booru tag autocompletion for A1111](https://github.com/DominikDoom/a1111-sd-webui-tagcomplete)
use_booru_tag_autocompletion_for_a1111 = False #@param {type: "boolean"}
if use_booru_tag_autocompletion_for_a1111:
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git {extension_dir}/tag-autocomplete

#@markdown ### ★ [ControlNet for Stable Diffusion WebUI](https://github.com/Mikubill/sd-webui-controlnet) ※上記のControl Net用のコードでダウンロード済みのためここでは処理しない
#@markdown ※生成する画像のポーズや構図を指定できる

#@markdown [使い方を確認する(YouTube)](https://)
# use_Control_Net = True #@param {type: "boolean"}
# if use_Control_Net:
#   !git clone https://github.com/Mikubill/sd-webui-controlnet.git {extension_dir}/sd-webui-controlnet

#@markdown ### ★ [Deforum Stable Diffusion — official extension for AUTOMATIC1111's webui](https://github.com/deforum-art/sd-webui-deforum)
use_sd_webui_deforum = False #@param {type: "boolean"}
if use_sd_webui_deforum:
  !git clone https://github.com/deforum-art/sd-webui-deforum.git /content/stable-diffusion-webui/extensions/deforum

#@markdown ### ★ [Detail Daemon](https://github.com/muerrilla/sd-webui-detail-daemon/)
use_detail_daemon = False #@param {type: "boolean"}
if use_detail_daemon:
  !git clone https://github.com/muerrilla/sd-webui-detail-daemon.git /content/stable-diffusion-webui/extensions/sd-webui-detail-daemon

#@markdown ### ★ [Dataset Tag Editor](https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor)
use_dataset_tag_editor = False #@param {type: "boolean"}
if use_dataset_tag_editor:
  !git clone https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-dataset-tag-editor

#@markdown ### ★ [Easy Generate Forever](https://github.com/blue-pen5805/sdweb-easy-generate-forever)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/e_LTlWbKTaE)
use_easy_generate_forever = False #@param {type: "boolean"}
if use_easy_generate_forever:
  !git clone https://github.com/blue-pen5805/sdweb-easy-generate-forever.git {extension_dir}/sdweb-easy-generate-forever

#@markdown ### ★ [Inpaint Anything for Stable Diffusion Web UI](https://github.com/Uminosachi/sd-webui-inpaint-anything)
#@markdown [公式ページ]
use_sd_webui_inpaint_anything = False #@param {type: "boolean"}
if use_sd_webui_inpaint_anything:
  # Inpaint Anything for Stable Diffusion Web UI
  !git clone https://github.com/Uminosachi/sd-webui-inpaint-anything.git /content/stable-diffusion-webui/extensions/sd-webui-inpaint-anything

#@markdown ### ★ [LoRA Block Weight](https://github.com/hako-mikan/sd-webui-lora-block-weight)
use_lora_block_weight = False #@param {type: "boolean"}
if use_lora_block_weight:
  !git clone https://github.com/hako-mikan/sd-webui-lora-block-weight.git /content/stable-diffusion-webui/extensions/sd-webui-lora-block-weight

#@markdown ### ★ [Mov2mov](https://github.com/Scholar01/sd-webui-mov2mov)
#@markdown [公式ページ]
use_mov2mov = False #@param {type: "boolean"}
if use_mov2mov:
  !git clone https://github.com/Scholar01/sd-webui-mov2mov.git /content/stable-diffusion-webui/extensions/sd-webui-mov2mov

#@markdown ### ★ [ReActor for Stable Diffusion](https://github.com/Gourieff/sd-webui-reactor)
#@markdown [公式ページ]
use_sd_webui_reactor = False #@param {type: "boolean"}
if use_sd_webui_reactor:
  !git clone https://github.com/Gourieff/sd-webui-reactor.git /content/stable-diffusion-webui/extensions/sd-webui-reactor

#@markdown ### ★ [Rembg](https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/nK9M7NYgZDI)
use_rembg = False #@param {type: "boolean"}
if use_rembg:
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg.git {extension_dir}/stable-diffusion-webui-rembg

#@markdown ### ★ [sd-civitai-browser (CivBrowser)](https://github.com/SignalFlagZ/sd-webui-civbrowser)
#@markdown [公式ページ]
use_sd_webui_civbrowser = False #@param {type: "boolean"}
if use_sd_webui_civbrowser:
  !git clone https://github.com/SignalFlagZ/sd-webui-civbrowser.git /content/stable-diffusion-webui/extensions/sd-webui-civbrowser

#@markdown ### ★ [sd_katanuki](https://github.com/aka7774/sd_katanuki)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/3WiiXMWierk)
use_sd_katanuki = False #@param {type: "boolean"}
if use_sd_katanuki:
  !git clone https://github.com/aka7774/sd_katanuki.git {extension_dir}/sd_katanuki

#@markdown ### ★ [sd-webui-depth-lib](https://github.com/jexom/sd-webui-depth-lib) ※手の修正
#@markdown [使い方を確認する(YouTube)](https://)
use_sd_webui_depth_lib = False #@param {type: "boolean"}
if use_sd_webui_depth_lib:
  !git clone https://github.com/jexom/sd-webui-depth-lib.git {extension_dir}/sd-webui-depth-lib

#@markdown ### ★ [sd-webui-enable-checker](https://github.com/shirayu/sd-webui-enable-checker)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/e_LTlWbKTaE)
use_sd_webui_enable_chacker = False #@param {type: "boolean"}
if use_sd_webui_enable_chacker:
  !git clone https://github.com/shirayu/sd-webui-enable-checker.git {extension_dir}/sd-webui-enable-checker

#@markdown ### ★ [sd-webui-freeu](https://github.com/ljleb/sd-webui-freeu)
use_sd_webui_freeu = False #@param {type: "boolean"}
if use_sd_webui_freeu:
  !git clone https://github.com/ljleb/sd-webui-freeu.git /content/stable-diffusion-webui/extensions/sd-webui-freeu

#@markdown ### ★ [SearchMyPNG](https://github.com/hina-choco/SearchMyPNG)
use_search_my_png = False #@param {type: "boolean"}
if use_search_my_png:
  !git clone https://github.com/hina-choco/SearchMyPNG.git /content/stable-diffusion-webui/extensions/SearchMyPNG

#@markdown ### ★ [Segment Anything for Stable Diffusion WebUI](https://github.com/continue-revolution/sd-webui-segment-anything)
use_sd_webui_segment_anything = False #@param {type: "boolean"}
if use_sd_webui_segment_anything:
  # Segment Anything for Stable Diffusion WebUI
  !git clone https://github.com/continue-revolution/sd-webui-segment-anything.git /content/stable-diffusion-webui/extensions/sd-webui-segment-anything
  # 必要なモデルファイルをダウンロード - Meta AI (Colab利用前提のためhuge, largeのみダウンロード)
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-segment-anything/models/sam
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth --directory-prefix=/content/stable-diffusion-webui/extensions/sd-webui-segment-anything/models/sam

#@markdown ### ★ [Stable Diffusion webui Infinite Image Browsing](https://github.com/zanllp/sd-webui-infinite-image-browsing)
use_sd_webui_infinite_image_browsing = False #@param {type: "boolean"}
if use_sd_webui_infinite_image_browsing:
  !git clone https://github.com/zanllp/sd-webui-infinite-image-browsing.git /content/stable-diffusion-webui/extensions/sd-webui-infinite-image-browsing

#@markdown ### ★ [SuperMerger](https://github.com/hako-mikan/sd-webui-supermerger)
use_super_merger = False #@param {type: "boolean"}
if use_super_merger:
  !git clone https://github.com/hako-mikan/sd-webui-supermerger.git /content/stable-diffusion-webui/extensions/sd-webui-supermerger

#@markdown ### ★ [TrainTrain](https://github.com/hako-mikan/sd-webui-traintrain)
use_traintrain = False #@param {type: "boolean"}
if use_traintrain:
  !git clone https://github.com/hako-mikan/sd-webui-traintrain.git /content/stable-diffusion-webui/extensions/sd-webui-traintrain

#@markdown ### ★ [Manga Editor Desu! Pro Edition](https://github.com/new-sankaku/stable-diffusion-webui-simple-manga-maker)
use_manga_editor_desu_pro_edition = False #@param {type: "boolean"}
if use_manga_editor_desu_pro_edition:
  !git clone https://github.com/new-sankaku/stable-diffusion-webui-simple-manga-maker.git /content/stable-diffusion-webui/extensions/stable-diffusion-webui-simple-manga-maker

print("Extensionのダウンロードが完了しました。")


# Extension（拡張機能）の情報
extension_choices = {
    "After Detailer":{
         "requirements_path": f"{extension_dir}/adetailer/requirements.txt"
    },
    "AnimateDiff":{
      "requirements_path": f"{extension_dir}/sd-webui-animatediff/requirements.txt"
    },
    "Booru tag autocompletion for A1111":{
          "requirements_path": f"{extension_dir}/tag-autocomplete/requirements.txt"
    },
    "sd-webui-controlnet":{
         "requirements_path": f"{extension_dir}/sd-webui-controlnet/requirements.txt"
    },
     "Deforum Stable Diffusion — official extension for AUTOMATIC1111's webui":{
          "requirements_path": f"{extension_dir}/deforum/requirements.txt"
    },
    "Detail Daemon":{
        "requirements_path": f"{extension_dir}/sd-webui-detail-daemon/requirements.txt"
    },
     "Dataset Tag Editor":{
          "requirements_path": f"{extension_dir}/stable-diffusion-webui-dataset-tag-editor/requirements.txt"
     },
     "Easy Generate Forever":{
          "requirements_path": f"{extension_dir}/sdweb-easy-generate-forever/requirements.txt"
    },
    "Inpaint Anything for Stable Diffusion Web UI":{
          "requirements_path": f"{extension_dir}/sd-webui-inpaint-anything/requirements.txt"
    },
    "LoRA Block Weight":{
          "requirements_path": f"{extension_dir}/sd-webui-lora-block-weight/requirements.txt"
    },
    "Mov2mov":{
          "requirements_path": f"{extension_dir}/sd-webui-mov2mov/requirements.txt"
    },
    "ReActor for Stable Diffusion":{
          "requirements_path": f"{extension_dir}/sd-webui-reactor/requirements.txt"
    },
    "stable-diffusion-webui-rembg":{
        "requirements_path": f"{extension_dir}/stable-diffusion-webui-rembg/requirements.txt"
    },
     "sd-civitai-browser (CivBrowser)":{
         "requirements_path":  f"{extension_dir}/sd-webui-civbrowser/requirements.txt"
    },
     "sd_katanuki":{
          "requirements_path": f"{extension_dir}/sd_katanuki/requirements.txt"
    },
    "sd-webui-depth-lib":{
        "requirements_path": f"{extension_dir}/sd-webui-depth-lib/requirements.txt"
    },
    "sd-webui-enable-checker":{
        "requirements_path": f"{extension_dir}/sd-webui-enable-checker/requirements.txt"
    },
     "sd-webui-freeu":{
          "requirements_path": f"{extension_dir}/sd-webui-freeu/requirements.txt"
     },
     "SearchMyPNG":{
           "requirements_path": f"{extension_dir}/SearchMyPNG/requirements.txt"
     },
    "sd-webui-segment-anything":{
        "requirements_path": f"{extension_dir}/sd-webui-segment-anything/requirements.txt"
    },
     "Stable Diffusion webui Infinite Image Browsing":{
           "requirements_path": f"{extension_dir}/sd-webui-infinite-image-browsing/requirements.txt"
     },
    "SuperMerger":{
          "requirements_path": f"{extension_dir}/sd-webui-supermerger/requirements.txt"
    },
     "TrainTrain":{
          "requirements_path": f"{extension_dir}/sd-webui-traintrain/requirements.txt"
     },
    "Manga Editor Desu! Pro Edition":{
            "requirements_path": f"{extension_dir}/stable-diffusion-webui-simple-manga-maker/requirements.txt"
    }
}

# 拡張機能のrequirements.txtをインストール
print("Extensionのrequirements.txtをインストール開始")
for key, file_info in extension_choices.items():
    if os.path.exists(file_info.get("requirements_path")):
        print(f"{key}の拡張機能のrequirements.txtをインストールしています...")
        try:
          !pip install -r {file_info.get("requirements_path")}
        except Exception as e:
          print(f"Error installing requirements for {key}: {e}")
          continue # エラーが発生しても次の拡張機能に進む
print(f"すべての処理が完全に完了しました！")

## stable-diffusion-webui起動

In [ ]:
#@title ### ★ 以下の "1. !python launch.py --share --xformers --enable-insecure-extension-access" を使用

# Web UIを実行
!python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **1. `!python launch.py --share --xformers --enable-insecure-extension-access`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。
    3. `--enable-insecure-extension-access`：セキュリティ制限を緩和して、全ての拡張機能にアクセスを許可。

- **最適な使用時：**  
  - **外部リンクで共有する必要があり、カスタム拡張機能を使用する場合。**
  - 信頼できる環境で、拡張機能をロードする際の制約を緩和する必要があるときに使用。  
    ※ セキュリティリスクが増すため、信頼できる拡張機能のみを使う場合に限ります。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。セキュリティ制限を緩和して拡張機能にアクセス可能にする
# !python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **2. `!python launch.py --share --xformers`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **外部リンクでWebUIを共有したい場合**。  
    Xformersを有効化してメモリ効率や生成速度を改善する目的で使用します。
  - 特に拡張機能のセキュリティ制限を緩和する必要がない場合に安全に使用可能。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。拡張機能のセキュリティ制限を緩和しない安全な方法
# !python launch.py --share --xformers

### **3. `!python /content/stable-diffusion-webui/launch.py --share --xformers`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **起動ディレクトリが不明確な場合や、明示的にスクリプトのフルパスを指定したい場合。**
  - Colab環境でディレクトリを移動している可能性がある場合に使用すると安心。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。外部共有とXformersを有効化
# !python /content/stable-diffusion-webui/launch.py --share --xformers

### **4. `!python /content/stable-diffusion-webui/launch.py --share`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
  - **Xformersは有効化されていません**。

- **最適な使用時：**  
  - **Xformersを必要とせず、単純にWebUIを共有したい場合。**
  - メモリ効率の最適化が不要な場合や、Xformersがインストールされていない環境での利用に適しています。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。Xformersを使用せず、外部共有のみを行う
# !python /content/stable-diffusion-webui/launch.py --share

### **その他. `!rm -rf /content/stable-diffusion-webui/tmp`**
- **注釈：**  
  - ディレクトリを完全に削除するコマンド。
  - キャッシュや一時ファイル（tempファイル）をクリアするために使用します。
  - 特定の不具合や古いキャッシュが原因でエラーが発生する場合に有効です。

- **最適な使用時：**  
  - Stable Diffusion WebUIを再起動する際に、不要な一時ファイルを削除してクリーンな状態にしたい場合。
  - モデルの切り替えや拡張機能を再インストールする際に、一時的なデータをクリアするために使用。

In [ ]:
# # キャッシュや一時ファイルを削除してクリーンな状態にする
# !rm -rf /content/stable-diffusion-webui/tmp

### **まとめ：コマンドの比較**
| 起動コマンド | 速度への影響 | 最適な使用状況 |
|----------|--------------|--------------|
| `!python launch.py --share --xformers --enable-insecure-extension-access` | **速い**（拡張機能次第でわずかに遅くなる場合あり）。 | 拡張機能を利用しつつ外部リンク共有する場合。 |
| `!python launch.py --share --xformers` | **最速**（拡張機能なし、効率化のみ）。 | 外部リンク共有が必要でXformersを有効化したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share --xformers` | **最速**（パス指定が違うだけで同じ挙動）。 | 安全に外部共有かつフルパス指定で確実に起動したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share` | **遅い**（`--xformers`がないため、計算効率が低下）。 | Xformersが不要な簡易な外部共有の場合。 |

---

| その他コマンド | 最適な使用状況 |
|----------|--------------|
| `!rm -rf /content/stable-diffusion-webui/tmp` | キャッシュクリアや不具合解消時に使用。 |

---
最も一般的で効率的なのは、**`!python launch.py --share --xformers`** です。必要に応じて拡張機能を使う場合は、`--enable-insecure-extension-access` を追加してください。

### **おすすめ**
- 拡張機能を使わない場合、**`!python launch.py --share --xformers`** を選択してください。
- 拡張機能を使用する場合、**`!python launch.py --share --xformers --enable-insecure-extension-access`** が最適です（速度を若干犠牲にして機能を拡張）。

## Reference
[safa-dayo/sd-webui-google-colab-setup.sh](https://gist.github.com/safa-dayo/b25f3737675c7667fefb51498dbf7212)